# Curve Fitting

*Martin Vonk (2025)*

It can be usefull to go from one soil model to the other. When the soil parameters are known the soil water retention curve and hydraulic conductivity function can be fitted.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import pedon as pe

In [ ]:
def plot_compare(soilsample: pe.SoilSample, soilmodel: pe.soilmodel.SoilModel):
    f, ax = plt.subplots(1, 2, sharey=True, figsize=(4.2, 4.5))
    ax[0].scatter(soilsample.theta, soilsample.h, c="k", s=10, label="Soil Sample")
    _ = pe.soilmodel.plot_swrc(
        soilmodel, ax=ax[0], label=f"Fitted Soil Model {soilmodel.__class__.__name__}"
    )
    ax[0].set_yscale("log")
    ax[0].set_xlim(0, 0.5)
    ax[0].set_yticks(soilsample.h)
    ax[0].set_xticks(np.linspace(0, 0.5, 6))

    ax[1].scatter(soilsample.k, soilsample.h, c="k", s=10)
    _ = pe.soilmodel.plot_hcf(soilmodel, ax=ax[1])

    ax[1].set_yscale("log")
    ax[1].set_xscale("log")

    k_left = 10 ** (np.floor(np.log10(min(soilsample.k))) - 1)
    k_right = 10 ** (np.ceil(np.log10(max(soilsample.k))) + 1)
    ax[1].set_xlim(k_left, k_right)
    ax[0].set_ylabel(r"|$\psi$| [cm]")
    ax[0].set_xlabel(r"$\theta$ [-]")
    ax[1].set_xlabel(r"$K_s$ [cm/d]")
    ncol = 3
    ax[0].legend(
        loc=(-0.02, 1),
        fontsize=6,
        frameon=False,
        ncol=ncol,
        columnspacing=0.8,
        handlelength=2.5,
    )

    f.align_xlabels()
    # plt.close(f)

In [ ]:
sn = "Sand"
soil = pe.Soil(sn).from_name(pe.Genuchten, "HYDRUS")
soilm_genuchten = getattr(soil, "model")
soilm_genuchten

In [ ]:
h = np.logspace(-4, 6, num=11)
k = soilm_genuchten.k(h)
theta = soilm_genuchten.theta(h)

In [ ]:
soilsample = pe.SoilSample(h=h, k=k, theta=theta)
soilsample

In [ ]:
soilm_panday = soilsample.fit(pe.Panday)
soilm_panday

In [ ]:
plot_compare(soilsample, soilm_panday)

The fit method finds the optimal curve through both the soil water retention curve and hydraulic conductivity function at the same time using the least squares algorithm. All parameters are subject to the optimization algorithm.

The fit method uses the optimization algorithm from the 1991 [RETC Code for Quantifying the Hydraulic Functions of Unsaturated Soils](https://www.pc-progress.com/Documents/programs/retc.pdf) by M.Th. van Genuchten, F.J. Leij and S.R. Yates.

The objective function $O(b)$ minimized is:

$ O(b) = \sum^N_{i=1}(w_i(\theta_{o,i}-\theta_i))^2 + \sum^M_{i=N+1}(w_iW_1W_2(Y_{o,i}-Y_i))^2$

Using the SciPy least-squares algorithm. 

With $N$ the number of $\theta$ data points and $M$ is the number of $K$ and $\theta$ data points

$w_i$ are the individual weight factors per measurements (by default 1 for each measurement)

$W_1$ is the weight factor for the hydraulic conductivity function with respect to the soil water retention (default is 0.1)

$W_2$ is the proportional weight factor for two different data types and elimination factor for different units. By default the formulation for 
$W_2 = \frac{(M-N)\sum^N_{i=1}w_i\theta_{o,i}}{N\sum^M_{i=N+1}w_i|Y_{o,i}|}$.

$Y$ is indicates the  for the logaritmic transform of such that $Y = log_{10}K$.

It can be favorable to only optimize the relative hydraulic conductivity function, and leave parameter k_s untouched. That kan be achieved by providing `k_s` to the fit method.

In [ ]:
soilm_panday = soilsample.fit(pe.Panday, k_s=max(soilsample.k))
soilm_panday

It is also possible to provide bounds for the parameter space. By default, the bounds argument is `None` which takes the stored parameter bounds per soil model:

In [ ]:
panday_bounds = pe.get_params("Panday")
panday_bounds

In [ ]:
panday_bounds.loc["k_s", "p_min"] = 650
panday_bounds.loc["k_s", "p_ini"] = max(soilsample.k)
soilm_panday = soilsample.fit(pe.Panday, pbounds=panday_bounds)
soilm_panday

Other available option are to print the optimization result.

In [ ]:
soilm_panday = soilsample.fit(pe.Panday, silent=False)

To parse other parameters for W1, W2, and individual weights there are other keyword arguements that for the fit method.